In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # put -1 to not use any

In [23]:
import pickle
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import cv2
import torch
import SimpleITK as sitk

from  PIL  import  Image
from lang_sam import LangSAM

#### sort csv by True/Predictd segmentation ratio

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

#### Crop eye region with LangSam

In [ ]:
CHECKPOINT_PATH='ckp/sam/sam_vit_h_4b8939.pth'
DATA_DIR = 'B images one eye/B one eye 2870 onwards'


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LangSAM(ckpt_path = CHECKPOINT_PATH)

In [ ]:
targets = []
crop_list = []
box_list = []

for img_name in os.listdir(DATA_DIR):
    
    img_path = os.path.join(DATA_DIR, img_name)
    targets.append(img_path)

save_dir = os.path.join(targets[0].split('B one eye 2870 onwards')[0], 'crop_mask')

i=0
text_prompt = 'eye'

for t in targets:
    print(f"Processing '{t}' : idx {i} / {len(targets)}")
    i+=1
    name = os.path.basename(t)

    np_im = cv2.imread(t)
    np_im = cv2.cvtColor(np_im, cv2.COLOR_BGR2RGB)

    h,w = np_im.shape[:2] 

    if np_im is None:
        print(f"Could not load '{t}' as an image, skipping...")
        continue
    out_path = os.path.join(save_dir, name)    
    im = Image.open(t)
    
    masks, boxes, labels, logits = model.predict(im, text_prompt)
    
    mask = masks[0]
    box = boxes[0]
    
    xmin,ymin, xmax, ymax = int(box[0].numpy()) -5, int(box[1].numpy()) -5 , int(box[2].numpy())+5 , int(box[3].numpy())+5
    
    
    cropped_img = np_im[ymin:ymax, xmin:xmax]

    crop_list.append(out_path)
    box_list.append(np.array([xmin, ymin, xmax, ymax]))


    cv2.imwrite(out_path,cropped_img)
    
    # plt.imshow(cropped_img)
    # plt.show()

failing cases -  eye closed, segmented other thing (i.e. nose), segmented nose or segmented eye of person in background

In [ ]:
df = pd.DataFrame(data={'img':targets, 'crop_img':crop_list, 'box':box_list})
df.to_csv('new_B_cropped_list.csv')